<a href="https://colab.research.google.com/github/Elish-Ab/Brent-Oil-Price-Prediction/blob/main/notebooks/Task_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install statsmodels arch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 733.0 kB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 985.3/985.3 kB 1.0 MB/s eta 0:00:001.1 MB/s eta 0:00:01

[notice] A new release of pip is available: 23.1.2 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.arima.model import ARIMA
from arch import arch_model

In [3]:
df = pd.read_csv('../data/Brent_Oil_Prices.csv')

FileNotFoundError: [Errno 2] No such file or directory: '../data/Brent_Oil_Prices.csv'

In [ ]:
df.head()

,Date,Price
0,20-May-87,18.63
1,21-May-87,18.45
2,22-May-87,18.55
3,25-May-87,18.60
4,26-May-87,18.63


In [4]:
df['Date'] = pd.to_datetime(df['Date'], infer_datetime_format=True)
df.set_index('Date', inplace=True)

# Check for missing values and handle them
df = df.dropna()

# Plot the data
plt.figure(figsize=(14, 7))
plt.plot(df['Price'], label='Brent Oil Price')
plt.title('Brent Oil Price Over Time')
plt.xlabel('Date')
plt.ylabel('Price (USD per Barrel)')
plt.legend()
plt.show()

NameError: name 'df' is not defined

In [5]:
# Perform the Augmented Dickey-Fuller test
result = adfuller(df['Price'])
print('ADF Statistic:', result[0])
print('p-value:', result[1])

# If the p-value > 0.05, difference the series
if result[1] > 0.05:
    df['Price_diff'] = df['Price'].diff().dropna()

NameError: name 'df' is not defined

In [6]:
# Difference the time series
df['Price_diff'] = df['Price'].diff().dropna()

# Plot the differenced series
plt.figure(figsize=(14, 7))
plt.plot(df['Price_diff'], label='Differenced Brent Oil Price')
plt.title('Differenced Brent Oil Price Over Time')
plt.xlabel('Date')
plt.ylabel('Price Difference (USD per Barrel)')
plt.legend()
plt.show()

# Re-run the Augmented Dickey-Fuller test
result_diff = adfuller(df['Price_diff'].dropna())
print('ADF Statistic after differencing:', result_diff[0])
print('p-value after differencing:', result_diff[1])

NameError: name 'df' is not defined

In [7]:
# Fit an ARIMA model
model = ARIMA(df['Price'], order=(1, 1, 1))  # (p, d, q) where d=1 as we've differenced once
model_fit = model.fit()

# Print model summary
print(model_fit.summary())

# Plot the model's fit
plt.figure(figsize=(14, 7))
plt.plot(df['Price'], label='Original Price')
plt.plot(model_fit.fittedvalues, color='red', label='Fitted Values')
plt.title('ARIMA Model Fit')
plt.xlabel('Date')
plt.ylabel('Price (USD per Barrel)')
plt.legend()
plt.show()


NameError: name 'df' is not defined